# IMDB 영화 평가

In [2]:
import numpy as np
import pandas as pd

## 1. 데이터 불러오기

In [8]:
df = pd.read_csv('labeledTrainData.tsv',sep='\t', quoting=3)
df.head(6)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
5,"""8196_8""",1,"""I dont know why people think this is such a b..."


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [10]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

## 2. 데이터 전처리

##  \<br>태그와 구두점 "" '' , 없애기

In [12]:
##  <br> 태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />',' ')

In [13]:
# 구둣점, 숫자 제거 --> 영어 이외의 문자는 공백으로 변환
import re

df['review'] = df. review.apply(lambda x: re.sub('[^A-Za-z]',' ',x)) 

In [14]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

## 3. train/ test 데이터셋 분리 및 텍스트 변환

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.review.values, df.sentiment, stratify = df.sentiment, random_state = 2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((18750,), (6250,), (18750,), (6250,))

### * countVectorizer

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [56]:
X_train_cv.shape, X_test_cv.shape

((18750, 1384106), (6250, 1384106))

### * LogisticRegression

In [38]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=300)
%time lr.fit(X_train_cv, y_train)

Wall time: 2min 14s


LogisticRegression(max_iter=300)

In [39]:
from sklearn.metrics import accuracy_score
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.8864

### * kNN

In [40]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
%time knn.fit(X_train_cv, y_train)

Wall time: 474 ms


KNeighborsClassifier()

In [41]:
from sklearn.metrics import accuracy_score
pred = knn.predict(X_test_cv)
accuracy_score(y_test, pred)

0.54736

### * TfidVectorizer

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvect.fit(X_train)
X_train_tf = tvect.transform(X_train)
X_test_tf = tvect.transform(X_test)

In [59]:
lr = LogisticRegression(max_iter=300)
%time lr.fit(X_train_tf, y_train)
pred_tf = lr.predict(X_test_tf)
accuracy_score(y_test, pred_tf)

Wall time: 55.7 s


0.87904

## 모델 저장하고 불러오기

In [46]:
import joblib

In [60]:
joblib.dump(tvect, 'model/imdb_tvect.pkl')
joblib.dump(lr, 'model/imdb_lr.pkl')

['model/imdb_lr.pkl']

In [61]:
# 변수제거
del tvect
del lr 

In [62]:
# 불러오기
new_tvect = joblib.load('model/imdb_tvect.pkl')
new_lr = joblib.load('model/imdb_lr.pkl')

In [63]:
new_X_test_tf = new_tvect.transform(X_test)
new_pred = new_lr.predict(new_X_test_tf)
accuracy_score(y_test, new_pred)

0.87904

## Pipeline과 GridSearchCV를 통한 하이퍼 파라메터 튜닝

In [64]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('tvect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr', LogisticRegression())
])
params = {
    'tvect__max_df': [100, 500],
    'lr__C': [1, 10]
}

In [65]:
from sklearn.model_selection import GridSearchCV
pipe_grid = GridSearchCV(
    pipeline, param_grid=params, cv=3,
    scoring='accuracy', verbose=1, n_jobs=-1
)
%time pipe_grid.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Wall time: 2min 58s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tvect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('lr', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'lr__C': [1, 10], 'tvect__max_df': [100, 500]},
             scoring='accuracy', verbose=1)

In [66]:
pipe_grid.best_score_

0.8802666666666666

In [67]:
pipe_grid.best_params_

{'lr__C': 10, 'tvect__max_df': 500}

In [68]:
pred = pipe_grid.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.88112

In [69]:
joblib.dump(pipe_grid, 'model/imdb_pipe.pkl')

['model/imdb_pipe.pkl']